# ATLAS RPV CNN Classifier

Mostly taken from Michela's Keras implementation here: https://github.com/mickypaganini/susy_rpv/blob/micky/train.py

In [1]:
import os

import h5py
import numpy as np

from keras import layers, models, callbacks
from keras import backend as K

Using TensorFlow backend.


## Global configuration

In [4]:
# Data config
n_train = 2**16 #412416
n_valid = 2**14 #137471
n_test = 2**14 #137471

## Load the data

In [5]:
input_dir = '/global/cscratch1/sd/sfarrell/atlas-rpv-images'

In [6]:
ls -lh $input_dir

total 1.8G
-rw-rw---- 1 sfarrell sfarrell 358M Apr 24 14:32 test.h5
-rw-rw---- 1 sfarrell sfarrell 1.1G Apr 24 14:32 train.h5
-rw-rw---- 1 sfarrell sfarrell 358M Apr 24 14:32 val.h5


In [7]:
def load_file(file, n_samples):
    print('Loading file', file)
    with h5py.File(file) as f:
        data_group = f['all_events']
        print('Loading %i/%i events' %
              (n_samples, data_group['hist'].shape[0]))
        data = data_group['hist'][:n_samples][:,:,:,None]
        labels = data_group['y'][:n_samples]
        weights = data_group['weight'][:n_samples]
    return data, labels, weights

In [8]:
%%time

train_file = os.path.join(input_dir, 'train.h5')
valid_file = os.path.join(input_dir, 'val.h5')
test_file = os.path.join(input_dir, 'test.h5')

train_input, train_labels, train_weights = load_file(train_file, n_train)
valid_input, valid_labels, valid_weights = load_file(valid_file, n_valid)
test_input, test_labels, test_weights = load_file(test_file, n_test)

Loading file /global/cscratch1/sd/sfarrell/atlas-rpv-images/train.h5
Loading 65536/412416 events
Loading file /global/cscratch1/sd/sfarrell/atlas-rpv-images/val.h5
Loading 16384/137471 events
Loading file /global/cscratch1/sd/sfarrell/atlas-rpv-images/test.h5
Loading 16384/137471 events
CPU times: user 18.7 s, sys: 1.32 s, total: 20 s
Wall time: 19.8 s


In [9]:
print('train shape:', train_input.shape)
print('valid shape:', valid_input.shape)
print('test shape: ', test_input.shape)

train shape: (65536, 64, 64, 1)
valid shape: (16384, 64, 64, 1)
test shape:  (16384, 64, 64, 1)


## Define the model

In [10]:
def build_model(input_shape, optimizer='adam'):
    inputs = layers.Input(shape=input_shape)
    h = layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1, padding='same')(inputs)
    h = layers.Conv2D(128, kernel_size=(3, 3), activation='relu', strides=2, padding='same')(h)
    h = layers.Conv2D(256, kernel_size=(3, 3), activation='relu', strides=2, padding='same')(h)
    h = layers.Conv2D(256, kernel_size=(3, 3), activation='relu', strides=2, padding='same')(h)
    h = layers.Flatten()(h)
    h = layers.Dense(256, activation='relu')(h)
    outputs = layers.Dense(1, activation='sigmoid')(h)
    model = models.Model(inputs, outputs, 'RPVClassifier')
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

## Train the model

In [11]:
# Model config

# Training config
batch_size = 256
n_epochs = 5

In [12]:
model = build_model(train_input.shape[1:])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 256)         590080    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               4194560   
__________

In [14]:
history = model.fit(train_input, train_labels,
                    sample_weight=train_weights,
                    batch_size=batch_size, epochs=n_epochs,
                    validation_data=(valid_input, valid_labels, valid_weights))

Train on 65536 samples, validate on 16384 samples
Epoch 1/5
65536/65536 [==============================] - 2294s - loss: 22521.0120 - acc: 0.5714 - val_loss: 3503.3587 - val_acc: 0.5796
Epoch 2/5
65536/65536 [==============================] - 2219s - loss: 3570.8338 - acc: 0.5715 - val_loss: 3503.3587 - val_acc: 0.5796
Epoch 3/5
65536/65536 [==============================] - 2176s - loss: 3570.8338 - acc: 0.5715 - val_loss: 3503.3587 - val_acc: 0.5796
Epoch 4/5
65536/65536 [==============================] - 2189s - loss: 3570.8338 - acc: 0.5715 - val_loss: 3503.3587 - val_acc: 0.5796
Epoch 5/5
65536/65536 [==============================] - 2201s - loss: 3570.8338 - acc: 0.5715 - val_loss: 3503.3587 - val_acc: 0.5796


## Evaluate on the test set

In [17]:
test_output = model.predict(test_input)